In [4]:
import apache_beam as beam

p1 = beam.Pipeline()

class ReusableTransform(beam.PTransform):
    def expand(self, pcol):
        calculate_and_format = (
            pcol
                | 'Calculate Sum' >> beam.Map(CalculateSum)
                | 'Apply formatting' >> beam.Map(FormatText)
        )
        return calculate_and_format
    
def SplitRow(input_elem):
    return input_elem.split(',')

def FilterByCountry(countryName, input_elem):
    return input_elem[1] == countryName

def CalculateSum(elem):
    return elem[0], (int(elem[2]) + int(elem[3]) + int(elem[4]))

def FormatText(elem):
    return f'{elem[0]} has received {elem[1]} marks'
    
input_collection = (
    p1
    | beam.io.ReadFromText('data/student_marks.txt')
    | beam.Map(SplitRow)
)

US_pipeline = (
    input_collection
    | beam.Filter(lambda record: FilterByCountry('US', record))
    # The step below must have a unique description in order to reuse
    | 'Call reusable PTransform for US' >> ReusableTransform()
    | 'Writing results to US File' >> beam.io.WriteToText('results/US_result')
)

India_pipeline = (
    input_collection
    | beam.Filter(lambda record: FilterByCountry('MX', record))
    # The step below must have a unique description in order to reuse
    | 'Call reusable PTransform for Mexico' >> ReusableTransform()
    | 'Writing results to Mexico File' >> beam.io.WriteToText('results/MX_result')
)

p1.run()
    

In [2]:
!cat results/MX_result-00000-of-00001

Carlos has received 83 marks
Santiago has received 93 marks


In [3]:
!cat results/US_result-00000-of-00001

John has received 82 marks
Jim has received 80 marks
